In [1]:
!pwd

/root/shared


In [2]:
!ls

Test_notebook.ipynb		    Untitled.ipynb	notes-FEniCSx.qmd
Test_tutorial_docker_version.ipynb  notes-FEniCSx.html	notes-FEniCSx_files


In [3]:
!which python3

/usr/bin/python3


In [4]:
!python3 --version

Python 3.10.6


In [5]:
import pyvista

ModuleNotFoundError: No module named 'pyvista'

In [6]:
!python3 -m pip install pyvista

  Using cached pyvista-0.42.2-py3-none-any.whl (1.6 MB)
  Using cached scooby-0.7.4-py3-none-any.whl (17 kB)
  Using cached pyvista-0.42.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.42.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.41.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.41.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.4-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.3-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.2-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.39.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.39.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.6-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.5-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.4-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.3-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.2-py3-none-any.whl (1.6

In [7]:
!python3 -m pip install vtk

ERROR: Could not find a version that satisfies the requirement vtk (from versions: none)
ERROR: No matching distribution found for vtk

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


From [Reconsider interactive visualization in notebooks and release Docker images #1378](https://github.com/FEniCS/dolfinx/issues/1378):

In [9]:
!python3 -m pip install 'jupyterlab>=3' ipywidgets 'pyvista[all,trame]'

  Using cached pyvista-0.42.2-py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.6 MB/s eta 0:00:00
  Using cached pyvista-0.42.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.42.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.41.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.41.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.4-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.3-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.2-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.40.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.39.1-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.39.0-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.6-py3-none-any.whl (1.6 MB)
  Using cached imageio-2.31.5-py3-none-any.whl (313 kB)
  Using cached pyvista-0.38.5-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0.38.4-py3-none-any.whl (1.6 MB)
  Using cached pyvista-0